In [1]:
import pandas as pd
import datetime
from re import sub
from decimal import Decimal
import io

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')

from langdetect import detect

from sklearn import preprocessing

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

# Change pandas viewing options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/danielacollaguazo/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Importing Data 

In [2]:
df_reviews = pd.read_csv("../../data/new-york-city-airbnb-open-data/reviews.csv")

## Sentiment Analysis of review comments

In [3]:
# df_reviews.shape
df_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2060,158,2008-09-22,2865,Thom,"very nice neighborhood,close enough to ""A"" tra..."
1,2595,17857,2009-11-21,50679,Jean,Notre séjour de trois nuits.\r\nNous avons app...
2,2595,19176,2009-12-05,53267,Cate,Great experience.
3,2595,19760,2009-12-10,38960,Anita,I've stayed with my friend at the Midtown Cast...
4,2595,34320,2010-04-09,71130,Kai-Uwe,"We've been staying here for about 9 nights, en..."


In [4]:
# calculates compound sentiment polarity of the sentence
vader_polarity_compound = lambda x: (SentimentIntensityAnalyzer().polarity_scores(x))['compound']

# We can retrieve scores for positive, negative or neutral sentiment. 
# We will use the compound: a normalized value: norm_score = score / math.sqrt((score * score) + alpha)
print(SentimentIntensityAnalyzer().polarity_scores('VADER is smart, handsome, and funny.'))

{'neg': 0.0, 'neu': 0.254, 'pos': 0.746, 'compound': 0.8316}


In [5]:
# determining the number of no comments
"{}% of reviews have empty comments".format((df_reviews.comments.isnull().sum() / df_reviews.comments.shape[0]) *100)
# "My name is {}, I'am {}".format("John",36)

'0.05879231633876933% of reviews have empty comments'

In [6]:
# Since the percentage is so insignificant, these rows will be dropped
df_reviews.dropna(subset=['comments'], how='any', axis=0, inplace=True)

Function that predicts the language. It needs to be passed a string with minimum amount of characters, thus the calculation on the fly of the lenght of the string passed.

In [7]:
def predict_lang(x):
    lang=''
    txt_len=len(x)
    if txt_len>50:
        try:
            lang=detect(x)
        except Exception as e:
            lang=''
    return lang

In [8]:
df_reviews['review_lang'] = df_reviews.comments.apply(lambda x: predict_lang(x))

Using the pre-trained Vader sentiment model based on NLTK go create polarity scores for all reviews:

In [9]:
df_reviews['polarity'] = df_reviews.comments.map(vader_polarity_compound)

We observe that a lot of reviews didnt get a language value. This is because their length was too short for the calculation

In [10]:
df_reviews.review_lang.value_counts(dropna=False)

en       1009590
          141117
fr         41587
es         38234
de         14021
it          6815
pt          5812
ko          2995
nl          2920
zh-cn       2664
ru          2113
ja          1610
da           839
sv           776
pl           339
no           332
ca           320
cs           288
fi           249
tr           121
hu            84
he            68
el            55
sk            37
af            37
vi            28
ro            28
th            27
zh-tw         24
ar            16
hr            15
so            11
sl            10
et            10
uk             8
bg             8
id             8
tl             4
cy             4
lt             2
sq             1
Name: review_lang, dtype: int64

Reviews were exported for both English and Spanish. I checked the spanish reviews fo sentiment and wasn't accurate what leads me to believe that this algorithm works best for English text

In [16]:
df_reviews.to_csv('output/reviews_with_sentiment_and_lang.csv', index=False)

## Turning df into a file-like object

In [15]:
# s_buf = io.StringIO()

# # text buffer
# df_reviews.to_csv(s_buf)

In [14]:
# to read the buffer
# s_buf.seek()